In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path_indata = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_indata.xlsx"
indata_sheet = "test_case_8_whNBD_poisson"
indataDF = pd.read_excel(excel_path_indata,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev
0,1,RDC,Johannesburg,1,40,1,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()

for id in indataDF.get(indataDF["Type"] == "Dealer").get("Installation id"):
    if indataDF.get(indataDF["Installation id"] == id).get("Demand type").to_numpy() == "Poisson":
        indataDF.loc[indataDF["Installation id"] == id, "Demand stdev"] = math.sqrt(indataDF.loc[outdataDF["Installation id"]== id, "Demand mean"].to_numpy())

sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()

Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [4]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance
0,1,RDC,Johannesburg,1,40,1,NaN,NBD,10.0,11.839608,NaN,10.0,11.839608,140.176307
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN,4.0,NaN,NaN,NaN
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN,25.0,NaN,NaN,NaN
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN,11.0,NaN,NaN,NaN
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN,37.0,NaN,NaN,NaN
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN,1.0,NaN,NaN,NaN
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN,3.0,NaN,NaN,NaN
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN,8.0,NaN,NaN,NaN
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN,45.0,NaN,NaN,NaN
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN,9.0,NaN,NaN,NaN


RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Shortage cost"] = p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.4304258274108533, betas are: [0.7865432  0.16102878 0.33212106 0.09337052 0.7030854  0.63211749
 0.45059161 0.09921657 0.447933   0.59825065]


In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdataDF.loc[outdataDF["Type"] == "RDC", "Q, subbatches"] = Q_0
outdataDF.loc[outdataDF["Type"] == "RDC", "R, subbatches"] = R_0
outdataDF


Starting optimizing, R = 0, c = 14.440024053315359, c+1 = 15.126498836591718
Doing downwards optimizing, R = -1, c = 13.783552605970804, c+1 = 14.440024053315359
Doing downwards optimizing, R = -2, c = 13.161703287965157, c+1 = 13.783552605970804
Doing downwards optimizing, R = -3, c = 12.574382929219697, c+1 = 13.161703287965157
Doing downwards optimizing, R = -4, c = 12.021490602534803, c+1 = 12.574382929219697
Doing downwards optimizing, R = -5, c = 11.50291696054421, c+1 = 12.021490602534803
Doing downwards optimizing, R = -6, c = 11.018543513580704, c+1 = 11.50291696054421
Doing downwards optimizing, R = -7, c = 10.56824184281785, c+1 = 11.018543513580704
Doing downwards optimizing, R = -8, c = 10.15187274245283, c+1 = 10.56824184281785
Doing downwards optimizing, R = -9, c = 9.769285284020878, c+1 = 10.15187274245283
Doing downwards optimizing, R = -10, c = 9.42031579517031, c+1 = 9.769285284020878
Doing downwards optimizing, R = -11, c = 9.104786744364944, c+1 = 9.42031579517031

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance,Shortage cost,Beta,"R, subbatches"
0,1,RDC,Johannesburg,1,40,1,NaN,NBD,10.0,11.839608,40.0,10.0,11.839608,140.176307,NaN,0.430426,-20.0
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN,4.0,NaN,NaN,NaN,19.0,0.786543,NaN
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN,25.0,NaN,NaN,NaN,19.0,0.161029,NaN
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN,11.0,NaN,NaN,NaN,19.0,0.332121,NaN
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN,37.0,NaN,NaN,NaN,19.0,0.093371,NaN
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN,1.0,NaN,NaN,NaN,19.0,0.703085,NaN
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN,3.0,NaN,NaN,NaN,19.0,0.632117,NaN
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN,8.0,NaN,NaN,NaN,19.0,0.450592,NaN
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN,45.0,NaN,NaN,NaN,19.0,0.099217,NaN
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN,9.0,NaN,NaN,NaN,19.0,0.447933,NaN


In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Lead time"] = indataDF.get(indataDF["Type"] == "RDC").get("Transport time")

In [8]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0*Q_subbatch_size # In units.
outdataDF.loc[outdataDF["Type"] == "RDC","Expected stock on hand"] = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected backorders"] = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)


Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0


In [9]:
# List comprehension for exp backorders.
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected backorders"] = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]

In [10]:
# Adding costs
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected holding costs per time unit"] = h_dealer_arr*np.array(exp_stock_on_hand_list)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected shortage costs per time unit"] = outdataDF.get(
    outdataDF["Type"] == "Dealer").get("Expected backorders").to_numpy()*p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected holding costs per time unit").to_numpy() + outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected shortage costs per time unit").to_numpy()

outdataDF.loc[outdataDF["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*outdataDF.get(outdataDF["Type"] == "RDC").get("Expected stock on hand").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "RDC").get("Expected holding costs per time unit").to_numpy()



In [11]:
excel_path_outdata = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_outdata.xlsx"
outdataDF.to_excel(excel_path_outdata,sheet_name = "Outdata_latest_testrun")

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,...,"R, subbatches",Waiting time,Lead time,R optimal,Realized item fill rate,Expected stock on hand,Expected backorders,Expected holding costs per time unit,Expected shortage costs per time unit,Total expected costs
0,1,RDC,Johannesburg,1,40,1,NaN,NBD,10.0,11.839608,...,-20.0,NaN,1.000000,-20.0,NaN,2.546749,12.046573,2.546749,NaN,2.546749
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,2.204657,4.0,0.973080,4.307503,0.012161,4.307503,0.231052,4.538556
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,4.204657,4.0,0.963943,12.831917,0.036575,12.831917,0.694919,13.526837
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,3.204657,4.0,0.963978,6.821057,0.025715,6.821057,0.488578,7.309636
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,2.204657,1.0,0.964461,17.825481,0.030138,17.825481,0.572627,18.398108
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,5.204657,9.0,0.960128,4.824846,0.029503,4.824846,0.560559,5.385405
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,4.204657,7.0,0.965477,4.818277,0.022934,4.818277,0.435753,5.254030
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,3.204657,4.0,0.950475,5.330699,0.035356,5.330699,0.671769,6.002468
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,6.204657,5.0,0.962953,21.850241,0.054898,21.850241,1.043060,22.893301
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN,...,NaN,1.204657,5.204657,7.0,0.965866,6.825207,0.029864,6.825207,0.567425,7.392632


In [12]:
beta_arr

array([0.7865432 , 0.16102878, 0.33212106, 0.09337052, 0.7030854 ,
       0.63211749, 0.45059161, 0.09921657, 0.447933  , 0.59825065])